In [7]:

from dotenv import dotenv_values
import weaviate
import weaviate.classes as wvc

env = dotenv_values()
wv_headers = {
    "X-OpenAI-Api-key": env["OPENAI_API_KEY"]
}
wv_client = weaviate.connect_to_local(headers=wv_headers)

scheme_chunk_collection = wv_client.collections.get("scheme_chunks")
scheme_whole_collection = wv_client.collections.get("whole_scheme")

user_input_dict = {
    "name": "GreenEats Food Service",
    "industry": "Food Service",
    "concern": "GreenEats Food Service is a mid-sized catering company in Singapore that specializes in providing healthy, sustainable meals for corporate events and private functions. However, with the increasing costs of energy, they are finding it challenging to maintain their profitability while upholding their commitment to sustainability.",
    # "concern": "Mrs. Tan is a retired elderly citizen living alone in Singapore. She has recently been diagnosed with a degenerative condition that affects her mobility and ability to perform daily tasks independently. With her condition expected to worsen over time, Mrs. Tan is concerned about the financial burden of long-term care and support, especially as she does not have any immediate family nearby to assist her.",
    "need": "GreenEats Food Service needs to upgrade its kitchen equipment to more energy-efficient alternatives to reduce their energy consumption and operational costs. They require modern cooking hobs, LED lighting, and refrigerators that can help them operate efficiently without compromising on the quality of their services.",
    "nature": "GreenEats Food Service operates a large commercial kitchen where they prepare a variety of dishes for events of different scales. Their kitchen runs round the clock to cater to various clients' demands. They rely heavily on equipment such as cooking hobs for food preparation, refrigerators for storage, and LED lighting for proper illumination. However, their current equipment is outdated and consumes a significant amount of energy, resulting in high utility bills. Upgrading to energy-efficient equipment would not only reduce their operational costs but also align with their brand ethos of sustainability.",
}

user_input = """
*Singaporean SME: GreenEats Food Service*
*Industry:* Food Service
*Concerns:* GreenEats Food Service is a mid-sized catering company in Singapore that specializes in providing healthy, sustainable meals for corporate events and private functions. However, with the increasing costs of energy, they are finding it challenging to maintain their profitability while upholding their commitment to sustainability.
*Needs:* GreenEats Food Service needs to upgrade its kitchen equipment to more energy-efficient alternatives to reduce their energy consumption and operational costs. They require modern cooking hobs, LED lighting, and refrigerators that can help them operate efficiently without compromising on the quality of their services.
*Nature of Work:* GreenEats Food Service operates a large commercial kitchen where they prepare a variety of dishes for events of different scales. Their kitchen runs round the clock to cater to various clients' demands. They rely heavily on equipment such as cooking hobs for food preparation, refrigerators for storage, and LED lighting for proper illumination. However, their current equipment is outdated and consumes a significant amount of energy, resulting in high utility bills. Upgrading to energy-efficient equipment would not only reduce their operational costs but also align with their brand ethos of sustainability.
"""

## Basic object search
# results = scheme_collection.query.fetch_objects(
#     return_properties=[
#         "title",
#         "content_chunks",
#         "source",
#     ],
# )
# print(results.objects[0].properties)

## Vector similarity search
rel_scheme = []
for k, v in user_input_dict.items():
    if k in ["name", "industry"]:
        continue
    results = scheme_chunk_collection.query.near_text(
        query=v,
        limit=2,
        target_vector="content_chunk",
        certainty=0.85,
        # distance=0.2,
        return_metadata=wvc.query.MetadataQuery(
            certainty=True,
            distance=True
        ),
    )
    for o in results.objects:
        # print(o.properties)
        # print(o.metadata.certainty)
        # print(o.metadata.distance)
        if o.properties["title"] not in rel_scheme:
            rel_scheme.append(o.properties["title"])

print(rel_scheme)

filter = wvc.query.Filter.by_property("title").contains_any(rel_scheme)
results = scheme_whole_collection.query.fetch_objects(
    filters=filter,
)
print(results.objects)
wv_client.close()

# results = scheme_collection.query.near_text(
#     query=user_input,
#     limit=3,
#     target_vector="content_chunk",
#     # certainty=0.85,
#     return_metadata=wvc.query.MetadataQuery(
#         certainty=True,
#         distance=True,
#     )
# )
# for o in results.objects:
#     print(o.properties)
#     print(o.metadata.certainty)
#     print(o.metadata.distance)
# wv_client.close()


['Energy Efficient Grant', 'Enterprise Development Grant']
[Object(uuid=_WeaviateUUIDInt('e32f1f8e-7f8a-4b74-9bf2-f6f1dac2c970'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'content': "# The Energy Efficient Grant\n\nThe Energy Efficient Grant (EEG) was launched in September 2022 and aims to help businesses within the Food and Service, Food Manufacturing or Retail sectors to cope with rising energy costs, through co-funding investments in more energy efficient equipment.\n\nIt provides financial support for SMEs to adopt pre-approval energy efficient equipment in the following categories: LED lighting, air-conditioning, cooking hobs, refrigerators, water heaters and clothes dryers.\n\n## EEG Eligibility\n\n### Who can apply?\nA SME can apply for EEG if they meet all the following criteria:\n- Is a business entity registerd and operating in Singapore


*Singaporean SME: GreenEats Food Service*

*Industry:* Food Service

*Concerns:* GreenEats Food Service is a mid-sized catering company in Singapore that specializes in providing healthy, sustainable meals for corporate events and private functions. However, with the increasing costs of energy, they are finding it challenging to maintain their profitability while upholding their commitment to sustainability.

*Needs:* GreenEats Food Service needs to upgrade its kitchen equipment to more energy-efficient alternatives to reduce their energy consumption and operational costs. They require modern cooking hobs, LED lighting, and refrigerators that can help them operate efficiently without compromising on the quality of their services.

*Nature of Work:* GreenEats Food Service operates a large commercial kitchen where they prepare a variety of dishes for events of different scales. Their kitchen runs round the clock to cater to various clients' demands. They rely heavily on equipment such as cooking hobs for food preparation, refrigerators for storage, and LED lighting for proper illumination. However, their current equipment is outdated and consumes a significant amount of energy, resulting in high utility bills. Upgrading to energy-efficient equipment would not only reduce their operational costs but also align with their brand ethos of sustainability.



**Singaporean Resident: Mrs. Tan**

**Concerns:** Mrs. Tan is a retired elderly citizen living alone in Singapore. She has recently been diagnosed with a degenerative condition that affects her mobility and ability to perform daily tasks independently. With her condition expected to worsen over time, Mrs. Tan is concerned about the financial burden of long-term care and support, especially as she does not have any immediate family nearby to assist her.

**Needs:** Mrs. Tan requires financial assistance to cover the costs of long-term care and support services as her condition progresses. She also needs access to specialized equipment and home modifications to ensure her safety and comfort at home. As her condition deteriorates, she may require professional caregivers or assistance with activities of daily living, such as bathing, dressing, and mobility.

**Nature of Work:** Mrs. Tan is a retired individual who used to work as a teacher. She leads a relatively independent lifestyle but now faces challenges due to her deteriorating health condition. She spends most of her time at home and relies on community services for occasional assistance with household chores and errands. However, as her condition worsens, she anticipates needing more comprehensive support and care to maintain her quality of life.
